<a href="https://colab.research.google.com/github/x1001000/yamnet-on-raspberrypi3/blob/main/colab_notebooks/IISNRL_labeled_by_MobileNetV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# if runtime reset

In [ ]:
!git clone https://github.com/qiuqiangkong/audioset_tagging_cnn.git
!mv audioset_tagging_cnn/* .

Cloning into 'audioset_tagging_cnn'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (370/370), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 699 (delta 206), reused 340 (delta 183), pack-reused 329
Receiving objects: 100% (699/699), 3.89 MiB | 1.54 MiB/s, done.
Resolving deltas: 100% (395/395), done.


In [ ]:
CHECKPOINT_PATH="MobileNetV1_mAP=0.389.pth"
!wget https://zenodo.org/record/3987831/files/MobileNetV1_mAP%3D0.389.pth?download=1 -O $CHECKPOINT_PATH

--2020-10-19 11:16:15--  https://zenodo.org/record/3987831/files/MobileNetV1_mAP%3D0.389.pth?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23639473 (23M) [application/octet-stream]
Saving to: ‘MobileNetV1_mAP=0.389.pth’

MobileNetV1_mAP=0.3 100%[===================>]  22.54M  6.40MB/s    in 3.5s    

2020-10-19 11:16:21 (6.40 MB/s) - ‘MobileNetV1_mAP=0.389.pth’ saved [23639473/23639473]



In [ ]:
!pip install torchlibrosa



# if runtime restart

In [ ]:
#import tensorflow as tf
#import tensorflow_hub as hub
import numpy as np
import csv
#import io

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import librosa
import scipy.signal

In [ ]:
#AudioSet Ontology
#import requests
#r = requests.get('https://raw.githubusercontent.com/audioset/ontology/master/ontology.json')
#len(r.json())

In [ ]:
def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

In [ ]:
# wav_file_name = 'speech_whistling2.wav'
# wav_file_name = 'test.wav'
def waveform(wav_file_name):
    #sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
    wav_data, sample_rate = librosa.load(wav_file_name)
    # stereo to mono
    if wav_data.ndim > 1:
        wav_data = wav_data[:,0]
    # normalization before resampling
    if wav_data.dtype == np.uint8:
        wav_data = wav_data / tf.uint8.max
    elif wav_data.dtype == np.int16:
        wav_data = wav_data / tf.int16.max
    elif wav_data.dtype == np.int32:
        wav_data = wav_data / tf.int32.max
    elif wav_data.dtype == np.float32:
        pass
    else:
        print('wav_data.dtype UNKNOWN')
    sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)

    # Show some basic information about the audio.
    #duration = len(wav_data)/sample_rate
    #print(f'Sample rate: {sample_rate} Hz')
    #print(f'Total duration: {duration:.2f}s')
    #print(f'Size of the input: {len(wav_data)}')

    # Let's listen to the wav file.
    #Audio(wav_data, rate=sample_rate)

    return wav_data

In [ ]:
def inference(waveform):
    with torch.no_grad():
        model.eval()
        batch_output_dict = model(waveform, None)

    clipwise_output = batch_output_dict['clipwise_output'].data.cpu().numpy()[0]
    """(classes_num,)"""

    sorted_indexes = np.argsort(clipwise_output)[::-1]

    return [np.array(labels)[sorted_indexes[k]] for k in range(3)]

In [ ]:
import sys
sys.path.append('pytorch')
sys.path.append('utils')
import librosa

from models import *
from pytorch_utils import move_data_to_device
import config

In [ ]:
MODEL_TYPE="MobileNetV1"
CHECKPOINT_PATH="MobileNetV1_mAP=0.389.pth"

sample_rate = 16000
window_size = 1024#400
hop_size = 320#160
mel_bins = 64
fmin = 50
fmax = 14000
model_type = MODEL_TYPE
checkpoint_path = CHECKPOINT_PATH
#audio_path = args.audio_path
device = torch.device('cpu')

classes_num = config.classes_num
labels = config.labels

Model = eval(model_type)
model = Model(sample_rate=sample_rate, window_size=window_size, 
        hop_size=hop_size, mel_bins=mel_bins, fmin=fmin, fmax=fmax, 
        classes_num=classes_num)

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


In [ ]:
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from os.path import isdir
import pandas as pd
df = pd.DataFrame(columns=['Filename','IISNRL','YAMNET #1','YAMNET #2','YAMNET #3'])
n = 0
for entry in os.scandir('/content/drive/My Drive/IISNRL1700'):
    if entry.is_dir():
        for wav_file_name in os.scandir(entry):
            if not wav_file_name.name.startswith('.'):
                n += 1
                print(n)#, wav_file_name.name)
                Waveform = waveform(wav_file_name.path)[None,:]
                Waveform = move_data_to_device(Waveform, None)
                try:
                    top1, top2, top3 = inference(Waveform)
                except:
                    print(wav_file_name.name)
                    df.loc[len(df)+1] = None
                    df.iloc[-1]['Filename'] = wav_file_name.name
                    df.iloc[-1]['IISNRL'] = 'Error: maybe wav too short!'
                else:
                    df.loc[len(df)+1] = None
                    df.iloc[-1]['Filename'] = wav_file_name.name
                    df.iloc[-1]['IISNRL'] = wav_file_name.name.split('_')[0]
                    df.iloc[-1]['YAMNET #1'] = top1
                    df.iloc[-1]['YAMNET #2'] = top2
                    df.iloc[-1]['YAMNET #3'] = top3
                #print(df)

In [ ]:
df.to_csv('/content/drive/My Drive/IISNRL1700_labeled_by_MobileNetV1.csv')